# DESI Fisher examples

In this notebook we will run some DESI forecasts.

In [1]:
import numpy as np

zedges = np.linspace(1.1, 1.6, 6)
zranges = list(zip(zedges[:-1], zedges[1:]))
nbars = [1079.9998, 1036.0009, 970.9995, 876.9994, 553.9985]  # nobj / deg^2 / dz
b0 = 0.84  # at z = 0

In [2]:
from cosmoprimo.fiducial import DESI
from desilike.theories.galaxy_clustering import (BAOPowerSpectrumTemplate, SimpleBAOWigglesTracerPowerSpectrumMultipoles, DampedBAOWigglesTracerPowerSpectrumMultipoles,
                                                 StandardPowerSpectrumTemplate, KaiserTracerPowerSpectrumMultipoles)
from desilike.observables.galaxy_clustering import TracerPowerSpectrumMultipolesObservable, CutskyFootprint, ObservablesCovarianceMatrix
from desilike.likelihoods.galaxy_clustering import ObservablesGaussianLikelihood, SNWeightedPowerSpectrumLikelihood
from desilike import Fisher, setup_logging

setup_logging()

## Let's compare simple isotropic BAO forecasts against some reference

### SN-weighted version

In [3]:
cosmo = DESI()
fo = cosmo.get_fourier()

covariances = []
for zrange, nbar in list(zip(zranges, nbars)):
    z = np.mean(zrange)
    r = 0.5
    sigmaper = (1 - r) * 12.4 * 0.758 * fo.sigma8_z(z, of='delta_cb') / 0.9
    f = fo.sigma8_z(z, of='theta_cb') / fo.sigma8_z(z, of='delta_cb')
    b1 = b0 * fo.sigma8_cb/fo.sigma8_z(z, of='delta_cb')  # prescription for linear bias
    params = {'b1': b1, 'sigmapar': (1. + f) * sigmaper, 'sigmaper': sigmaper}
    # Footprint, to get volume and shot noise; provided nbar is in deg^(-2) dz^(-1)
    footprint = CutskyFootprint(area=14000., zrange=zrange, nbar=nbar * np.diff(zrange), cosmo=cosmo)
    template = BAOPowerSpectrumTemplate(z=z, fiducial='DESI', apmode='qisoqap')
    #theory = DampedBAOWigglesTracerPowerSpectrumMultipoles(template=template)
    theory = SimpleBAOWigglesTracerPowerSpectrumMultipoles(template=template)
    likelihood = SNWeightedPowerSpectrumLikelihood(theories=theory, data=params, footprints=footprint, klim=(0.01, 0.5))

    fisher = Fisher(likelihood)
    precision = fisher(**params)
    covariances.append(precision.to_covariance(params=['qiso']))

# Let's compare against reference: within a few percent, not too bad!
refs = [0.7019, 0.7107, 0.7296, 0.7658, 1.0062]
print(100. * np.array([cov.std(params='qiso') for cov in covariances]) / np.array(refs)[:len(covariances)])

[000000.69] [0/1] 01-31 09:47  absl                      INFO     Unable to initialize backend 'tpu_driver': NOT_FOUND: Unable to find driver in registry given worker: 
[000000.69] [0/1] 01-31 09:47  absl                      INFO     Unable to initialize backend 'cuda': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
[000000.69] [0/1] 01-31 09:47  absl                      INFO     Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
[000000.70] [0/1] 01-31 09:47  absl                      INFO     Unable to initialize backend 'tpu': INVALID_ARGUMENT: TpuPlatform is not available.
[000000.70] [0/1] 01-31 09:47  absl                      WARNING  No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


/home/adematti/Bureau/DESI/NERSC/cosmodesi/cosmoprimo/cosmoprimo/eisenstein_hu.py:23: UserWarning: EisensteinHuNoWiggleEngine cannot cope with massive neutrinos
  warnings.warn('{} cannot cope with massive neutrinos'.format(self.__class__.__name__))


[000001.73] [0/1] 01-31 09:47  Differentiation           INFO     Varied parameters: ['qiso', 'qap', 'b1', 'al0_-3', 'al0_-2', 'al0_-1', 'al0_0', 'al0_1', 'al2_-3', 'al2_-2', 'al2_-1', 'al2_0', 'al2_1'].
[000001.88] [0/1] 01-31 09:47  Differentiation           INFO     Using auto-differentiation for parameter qiso.
[000001.91] [0/1] 01-31 09:47  Differentiation           INFO     Using auto-differentiation for parameter qap.
[000001.93] [0/1] 01-31 09:47  Differentiation           INFO     Using auto-differentiation for parameter b1.
[000001.96] [0/1] 01-31 09:47  Differentiation           INFO     Using auto-differentiation for parameter al0_-3.
[000001.99] [0/1] 01-31 09:47  Differentiation           INFO     Using auto-differentiation for parameter al0_-2.
[000002.02] [0/1] 01-31 09:47  Differentiation           INFO     Using auto-differentiation for parameter al0_-1.
[000002.05] [0/1] 01-31 09:47  Differentiation           INFO     Using auto-differentiation for parameter al0_0.
[

[000009.58] [0/1] 01-31 09:47  Differentiation           INFO     Using auto-differentiation for parameter al0_1.
[000009.61] [0/1] 01-31 09:47  Differentiation           INFO     Using auto-differentiation for parameter al2_-3.
[000009.65] [0/1] 01-31 09:47  Differentiation           INFO     Using auto-differentiation for parameter al2_-2.
[000009.68] [0/1] 01-31 09:47  Differentiation           INFO     Using auto-differentiation for parameter al2_-1.
[000009.70] [0/1] 01-31 09:47  Differentiation           INFO     Using auto-differentiation for parameter al2_0.
[000009.73] [0/1] 01-31 09:47  Differentiation           INFO     Using auto-differentiation for parameter al2_1.
[000009.76] [0/1] 01-31 09:47  Differentiation           INFO     Varied parameters: ['qiso', 'qap', 'b1', 'al0_-3', 'al0_-2', 'al0_-1', 'al0_0', 'al0_1', 'al2_-3', 'al2_-2', 'al2_-1', 'al2_0', 'al2_1'].
[000009.80] [0/1] 01-31 09:47  Differentiation           INFO     Using finite-differentiation for parameter 

[000015.20] [0/1] 01-31 09:47  Differentiation           INFO     Using finite-differentiation for parameter qap.
[000015.30] [0/1] 01-31 09:47  Differentiation           INFO     Using auto-differentiation for parameter b1.
[000015.39] [0/1] 01-31 09:47  Differentiation           INFO     Using auto-differentiation for parameter al0_-3.
[000015.49] [0/1] 01-31 09:47  Differentiation           INFO     Using auto-differentiation for parameter al0_-2.
[000015.57] [0/1] 01-31 09:47  Differentiation           INFO     Using auto-differentiation for parameter al0_-1.
[000015.64] [0/1] 01-31 09:47  Differentiation           INFO     Using auto-differentiation for parameter al0_0.
[000015.70] [0/1] 01-31 09:47  Differentiation           INFO     Using auto-differentiation for parameter al0_1.
[000015.78] [0/1] 01-31 09:47  Differentiation           INFO     Using auto-differentiation for parameter al2_-3.
[000015.87] [0/1] 01-31 09:47  Differentiation           INFO     Using auto-differenti

### Comparison against multipole compression

In [4]:
covariances2 = []
for zrange, nbar in list(zip(zranges, nbars)):
    z = np.mean(zrange)
    r = 0.5
    sigmaper = (1 - r) * 12.4 * 0.758 * fo.sigma8_z(z, of='delta_cb') / 0.9
    f = fo.sigma8_z(z, of='theta_cb') / fo.sigma8_z(z, of='delta_cb')
    params = {'b1': b0 * fo.sigma8_cb/fo.sigma8_z(z, of='delta_cb'), 'sigmapar': (1. + f) * sigmaper, 'sigmaper': sigmaper}
    
    footprint = CutskyFootprint(area=14000., zrange=zrange, nbar=nbar * np.diff(zrange), cosmo=cosmo)
    template = BAOPowerSpectrumTemplate(z=z, fiducial='DESI', apmode='qisoqap')
    #theory = DampedBAOWigglesTracerPowerSpectrumMultipoles(template=template)
    theory = SimpleBAOWigglesTracerPowerSpectrumMultipoles(template=template)
    observable = TracerPowerSpectrumMultipolesObservable(data=params,  # data can be a dictionary of parameters
                                                         # fit monopole and quadrupole, between 0.02 and 0.5 h/Mpc, with 0.005 h/Mpc steps
                                                         klim={0: [0.01, 0.5, 0.005], 2: [0.01, 0.5, 0.005], 4: [0.01, 0.5, 0.005]},
                                                         theory=theory)
    covariance = ObservablesCovarianceMatrix(observable, footprints=footprint, resolution=5)  # Gaussian covariance matrix
    likelihood = ObservablesGaussianLikelihood(observables=observable, covariance=covariance(**params))
    fisher = Fisher(likelihood)
    precision = fisher(**params)
    covariances2.append(precision.to_covariance(params=['qiso']))

# multipole compression is very close to optimal...
print(np.array([cov.std(params='qiso') for cov in covariances2]) / np.array([cov.std(params='qiso') for cov in covariances]))

/home/adematti/anaconda3/envs/cosmodesi/lib/python3.9/site-packages/numpy/lib/polynomial.py:1329: FutureWarning: In the future extra properties will not be copied across when constructing one poly1d from another
  other = poly1d(other)


[000025.65] [0/1] 01-31 09:48  Differentiation           INFO     Varied parameters: ['qiso', 'qap', 'b1', 'al0_-3', 'al0_-2', 'al0_-1', 'al0_0', 'al0_1', 'al2_-3', 'al2_-2', 'al2_-1', 'al2_0', 'al2_1', 'al4_-3', 'al4_-2', 'al4_-1', 'al4_0', 'al4_1'].
[000025.68] [0/1] 01-31 09:48  Differentiation           INFO     Using auto-differentiation for parameter qiso.
[000025.72] [0/1] 01-31 09:48  Differentiation           INFO     Using auto-differentiation for parameter qap.
[000025.75] [0/1] 01-31 09:48  Differentiation           INFO     Using auto-differentiation for parameter b1.
[000025.79] [0/1] 01-31 09:48  Differentiation           INFO     Using auto-differentiation for parameter al0_-3.
[000025.82] [0/1] 01-31 09:48  Differentiation           INFO     Using auto-differentiation for parameter al0_-2.
[000025.86] [0/1] 01-31 09:48  Differentiation           INFO     Using auto-differentiation for parameter al0_-1.
[000025.89] [0/1] 01-31 09:48  Differentiation           INFO     U

[000038.20] [0/1] 01-31 09:48  Differentiation           INFO     Using auto-differentiation for parameter al0_1.
[000038.24] [0/1] 01-31 09:48  Differentiation           INFO     Using auto-differentiation for parameter al2_-3.
[000038.29] [0/1] 01-31 09:48  Differentiation           INFO     Using auto-differentiation for parameter al2_-2.
[000038.34] [0/1] 01-31 09:48  Differentiation           INFO     Using auto-differentiation for parameter al2_-1.
[000038.38] [0/1] 01-31 09:48  Differentiation           INFO     Using auto-differentiation for parameter al2_0.
[000038.43] [0/1] 01-31 09:48  Differentiation           INFO     Using auto-differentiation for parameter al2_1.
[000038.47] [0/1] 01-31 09:48  Differentiation           INFO     Using auto-differentiation for parameter al4_-3.
[000038.52] [0/1] 01-31 09:48  Differentiation           INFO     Using auto-differentiation for parameter al4_-2.
[000038.56] [0/1] 01-31 09:48  Differentiation           INFO     Using auto-differ

[000057.13] [0/1] 01-31 09:48  Differentiation           INFO     Using auto-differentiation for parameter al4_-1.
[000057.17] [0/1] 01-31 09:48  Differentiation           INFO     Using auto-differentiation for parameter al4_0.
[000057.20] [0/1] 01-31 09:48  Differentiation           INFO     Using auto-differentiation for parameter al4_1.
[000057.22] [0/1] 01-31 09:48  Differentiation           INFO     Varied parameters: ['qiso', 'qap', 'b1', 'al0_-3', 'al0_-2', 'al0_-1', 'al0_0', 'al0_1', 'al2_-3', 'al2_-2', 'al2_-1', 'al2_0', 'al2_1', 'al4_-3', 'al4_-2', 'al4_-1', 'al4_0', 'al4_1'].
[000057.25] [0/1] 01-31 09:48  Differentiation           INFO     Using finite-differentiation for parameter qiso.
[000057.28] [0/1] 01-31 09:48  Differentiation           INFO     Using finite-differentiation for parameter qap.
[000057.32] [0/1] 01-31 09:48  Differentiation           INFO     Using auto-differentiation for parameter b1.
[000057.37] [0/1] 01-31 09:48  Differentiation           INFO    

## BAO + full shape forecasts

In [5]:
cosmo = DESI()
fo = cosmo.get_fourier()

covariances_bao, covariances_bao_highk, covariances_fs_lowk, covariances_bao_fs = [], [], [], []
for zrange, nbar in list(zip(zranges, nbars)):
    z = np.mean(zrange)
    r = 0.5
    sigmaper = (1 - r) * 12.4 * 0.758 * fo.sigma8_z(z, of='delta_cb') / 0.9
    f = fo.sigma8_z(z, of='theta_cb') / fo.sigma8_z(z, of='delta_cb')
    b1 = b0 * fo.sigma8_cb/fo.sigma8_z(z, of='delta_cb')
    params = {'b1': b1, 'sigmapar': (1. + f) * sigmaper, 'sigmaper': sigmaper}
    footprint = CutskyFootprint(area=14000., zrange=zrange, nbar=nbar * np.diff(zrange), cosmo=cosmo)
    template = BAOPowerSpectrumTemplate(z=z, fiducial='DESI', apmode='qparqper')
    #theory = DampedBAOWigglesTracerPowerSpectrumMultipoles(template=template)
    theory = SimpleBAOWigglesTracerPowerSpectrumMultipoles(template=template)
    #for param in theory.params.select(basename='al*'): param.update(fixed=True)
    likelihood = SNWeightedPowerSpectrumLikelihood(theories=theory, data=params, footprints=footprint, klim=(0.01, 0.5))
    fisher = Fisher(likelihood)
    precision_bao = fisher(**params)
    covariances_bao.append(precision_bao.to_covariance(params=['qpar', 'qper']))
    
    likelihood.init.update(klim=(0.1, 0.5))
    fisher = Fisher(likelihood)
    precision_bao = fisher(**params)
    covariances_bao_highk.append(precision_bao.to_covariance(params=['qpar', 'qper']))
    
    #likelihood.init.update(klim=(0.01, 0.1))
    params = {'b1': b1}
    template = StandardPowerSpectrumTemplate(z=z, fiducial='DESI', apmode='qparqper')
    theory = KaiserTracerPowerSpectrumMultipoles(template=template)
    likelihood = SNWeightedPowerSpectrumLikelihood(theories=theory, data=params, footprints=footprint, klim=(0.01, 0.1))
    fisher = Fisher(likelihood)
    precision_fs = fisher(**params)
    covariances_fs_lowk.append(precision_fs.to_covariance(params=['qpar', 'qper']))
    covariances_bao_fs.append((precision_bao + precision_fs).to_covariance(params=['qpar', 'qper', 'f']))

[000068.50] [0/1] 01-31 09:48  Differentiation           INFO     Varied parameters: ['qpar', 'qper', 'b1', 'al0_-3', 'al0_-2', 'al0_-1', 'al0_0', 'al0_1', 'al2_-3', 'al2_-2', 'al2_-1', 'al2_0', 'al2_1'].
[000068.54] [0/1] 01-31 09:48  Differentiation           INFO     Using auto-differentiation for parameter qpar.
[000068.57] [0/1] 01-31 09:48  Differentiation           INFO     Using auto-differentiation for parameter qper.
[000068.60] [0/1] 01-31 09:48  Differentiation           INFO     Using auto-differentiation for parameter b1.
[000068.63] [0/1] 01-31 09:48  Differentiation           INFO     Using auto-differentiation for parameter al0_-3.
[000068.65] [0/1] 01-31 09:48  Differentiation           INFO     Using auto-differentiation for parameter al0_-2.
[000068.67] [0/1] 01-31 09:48  Differentiation           INFO     Using auto-differentiation for parameter al0_-1.
[000068.70] [0/1] 01-31 09:48  Differentiation           INFO     Using auto-differentiation for parameter al0_0.

[000073.58] [0/1] 01-31 09:48  Differentiation           INFO     Using auto-differentiation for parameter f.
[000073.61] [0/1] 01-31 09:48  Differentiation           INFO     Using auto-differentiation for parameter b1.
[000073.69] [0/1] 01-31 09:48  Differentiation           INFO     Using auto-differentiation for parameter sn0.


/home/adematti/Bureau/DESI/NERSC/cosmodesi/desilike/desilike/parameter.py:2764: RuntimeWarning: Mean of empty slice
  new._center = np.nanmean(centers, axis=0)


[000075.90] [0/1] 01-31 09:48  Differentiation           INFO     Varied parameters: ['qpar', 'qper', 'b1', 'al0_-3', 'al0_-2', 'al0_-1', 'al0_0', 'al0_1', 'al2_-3', 'al2_-2', 'al2_-1', 'al2_0', 'al2_1'].
[000075.93] [0/1] 01-31 09:48  Differentiation           INFO     Using auto-differentiation for parameter qpar.
[000075.96] [0/1] 01-31 09:48  Differentiation           INFO     Using auto-differentiation for parameter qper.
[000075.99] [0/1] 01-31 09:48  Differentiation           INFO     Using auto-differentiation for parameter b1.
[000076.01] [0/1] 01-31 09:48  Differentiation           INFO     Using auto-differentiation for parameter al0_-3.
[000076.04] [0/1] 01-31 09:48  Differentiation           INFO     Using auto-differentiation for parameter al0_-2.
[000076.06] [0/1] 01-31 09:48  Differentiation           INFO     Using auto-differentiation for parameter al0_-1.
[000076.08] [0/1] 01-31 09:48  Differentiation           INFO     Using auto-differentiation for parameter al0_0.

[000079.94] [0/1] 01-31 09:48  Differentiation           INFO     Using auto-differentiation for parameter f.
[000079.98] [0/1] 01-31 09:48  Differentiation           INFO     Using auto-differentiation for parameter b1.
[000080.01] [0/1] 01-31 09:48  Differentiation           INFO     Using auto-differentiation for parameter sn0.
[000080.66] [0/1] 01-31 09:48  Differentiation           INFO     Varied parameters: ['qpar', 'qper', 'b1', 'al0_-3', 'al0_-2', 'al0_-1', 'al0_0', 'al0_1', 'al2_-3', 'al2_-2', 'al2_-1', 'al2_0', 'al2_1'].
[000080.70] [0/1] 01-31 09:48  Differentiation           INFO     Using auto-differentiation for parameter qpar.
[000080.74] [0/1] 01-31 09:48  Differentiation           INFO     Using auto-differentiation for parameter qper.
[000080.77] [0/1] 01-31 09:48  Differentiation           INFO     Using auto-differentiation for parameter b1.
[000080.80] [0/1] 01-31 09:48  Differentiation           INFO     Using auto-differentiation for parameter al0_-3.
[000080.83

[000084.99] [0/1] 01-31 09:48  Differentiation           INFO     Varied parameters: ['qpar', 'qper', 'f', 'b1', 'sn0'].
[000085.03] [0/1] 01-31 09:48  Differentiation           INFO     Using auto-differentiation for parameter qpar.
[000085.10] [0/1] 01-31 09:48  Differentiation           INFO     Using auto-differentiation for parameter qper.
[000085.15] [0/1] 01-31 09:48  Differentiation           INFO     Using auto-differentiation for parameter f.
[000085.20] [0/1] 01-31 09:48  Differentiation           INFO     Using auto-differentiation for parameter b1.
[000085.26] [0/1] 01-31 09:48  Differentiation           INFO     Using auto-differentiation for parameter sn0.
[000086.00] [0/1] 01-31 09:49  Differentiation           INFO     Varied parameters: ['qpar', 'qper', 'b1', 'al0_-3', 'al0_-2', 'al0_-1', 'al0_0', 'al0_1', 'al2_-3', 'al2_-2', 'al2_-1', 'al2_0', 'al2_1'].
[000086.04] [0/1] 01-31 09:49  Differentiation           INFO     Using auto-differentiation for parameter qpar.
[0

[000090.38] [0/1] 01-31 09:49  Differentiation           INFO     Using auto-differentiation for parameter f.
[000090.39] [0/1] 01-31 09:49  Differentiation           INFO     Using auto-differentiation for parameter b1.
[000090.41] [0/1] 01-31 09:49  Differentiation           INFO     Using auto-differentiation for parameter sn0.
[000090.44] [0/1] 01-31 09:49  Differentiation           INFO     Varied parameters: ['qpar', 'qper', 'f', 'b1', 'sn0'].
[000090.49] [0/1] 01-31 09:49  Differentiation           INFO     Using auto-differentiation for parameter qpar.
[000090.54] [0/1] 01-31 09:49  Differentiation           INFO     Using auto-differentiation for parameter qper.
[000090.58] [0/1] 01-31 09:49  Differentiation           INFO     Using auto-differentiation for parameter f.
[000090.62] [0/1] 01-31 09:49  Differentiation           INFO     Using auto-differentiation for parameter b1.
[000090.66] [0/1] 01-31 09:49  Differentiation           INFO     Using auto-differentiation for pa

[000095.86] [0/1] 01-31 09:49  Differentiation           INFO     Varied parameters: ['qpar', 'qper', 'f', 'b1', 'sn0'].
[000095.88] [0/1] 01-31 09:49  Differentiation           INFO     Using auto-differentiation for parameter qpar.
[000095.89] [0/1] 01-31 09:49  Differentiation           INFO     Using auto-differentiation for parameter qper.
[000095.91] [0/1] 01-31 09:49  Differentiation           INFO     Using auto-differentiation for parameter f.
[000095.93] [0/1] 01-31 09:49  Differentiation           INFO     Using auto-differentiation for parameter b1.
[000095.95] [0/1] 01-31 09:49  Differentiation           INFO     Using auto-differentiation for parameter sn0.
[000095.97] [0/1] 01-31 09:49  Differentiation           INFO     Varied parameters: ['qpar', 'qper', 'f', 'b1', 'sn0'].
[000096.03] [0/1] 01-31 09:49  Differentiation           INFO     Using auto-differentiation for parameter qpar.
[000096.07] [0/1] 01-31 09:49  Differentiation           INFO     Using auto-different

In [6]:
# 5% - 15% improvement using the full shape < 0.1 h/Mpc
for cov_bao, cov_bao_fs in zip(covariances_bao, covariances_bao_fs):
    print(cov_bao.std('qpar') / cov_bao_fs.std('qpar'), cov_bao.std('qper') / cov_bao_fs.std('qper'))

1.1338654693290975 1.0544353975346212
1.132756400752436 1.0530683141540027
1.1330648924122124 1.0525984879390842
1.1358553999979024 1.0537128951487504
1.1655466609561944 1.0730626196827129
